In [0]:
import emoji
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Sequential

In [0]:
train=pd.read_csv("/content/train_emoji.csv",header=None)
test=pd.read_csv("/content/test_emoji.csv",header=None)

In [0]:
train.head()
#Ist columns shows senetence and second the label

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [0]:
# emoji.EMOJI_ALIAS_UNICODE  shows all the emojis and its name

In [0]:
emoji_dictionary={
    "0":"\u2764\uFE0F",
     "1": ":baseball:",
      "2": ":beaming_face_with_smiling_eyes:",
      "3": ":downcast_face_with_sweat:",
      "4": ":fork_and_knife:"
}

In [0]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [0]:
#emojis we will used in our data
for e in emoji_dictionary.values():
  print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [0]:
data=train.values
for i in range(10):
  print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [0]:
XT=train[0]
Xt=test[0]
YT=to_categorical(train[1])
Yt=to_categorical(test[1])
print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [0]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
embeddings={}
count=0
with open('/content/glove.6B.50d.txt',encoding='utf-8')as f:
  for line in f:
      values=line.split()
      word=values[0]
      coeff=np.asarray(values[1:],dtype='float32')
      count=count+1
      if(count<=2):
        print(word,coeff) #display two words from glove
      embeddings[word]=coeff  

the [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]
, [ 0.013441  0.23682  -0.16899   0.40951   0.63812   0.47709  -0.42852
 -0.55641  -0.364    -0.23938   0.13001  -0.063734 -0.39575  -0.48162
  0.23291   0.090201 -0.13324   0.078639 -0.41634  -0.15428   0.10068
  0.48891   0.31226  -0.1252   -0.037512 -1.5179    0.12612  -0.02442
 -0.042961 -0.28351   3.5416   -0.11956  -0.014533 -0.1499    0.21864
 -0.33412  -0.13872   0.31806   0.

In [0]:
#so above you can see word with 50 coeff
#so here we want make our embedding layer of size 135(no of lines),10(no of words),50(words embedding size)
def getOutputEmbeddings(X):
  embedding_matrix_output=np.zeros((X.shape[0],10,50))
  for ix in range(X.shape[0]):
    X[ix]=X[ix].split() # so our next loop iterate over every word
    for jx in range(len(X[ix])):
      embedding_matrix_output[ix][jx]=embeddings[X[ix][jx].lower()]
  return embedding_matrix_output

In [0]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

In [0]:
print(emb_XT.shape)
print(emb_Xt.shape)


(132, 10, 50)
(56, 10, 50)


In [0]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

In [0]:
model.evaluate(emb_Xt,Yt)

56/56 [==============================] - 0s 298us/step


[1.1397899900163924, 0.6785714030265808]

In [0]:
pred = model.predict_classes(emb_Xt)

In [0]:
#testing the predictions
for i in range(5):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
❤️
she got me a present
❤️
❤️
ha ha ha it was so funny
😁
😁
